In [1]:
# Check for GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: False
GPU: None


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Extract title, subtitle, keywords and comments for some sample articles and
# store them
import os
import json
import requests

samples_dir = '/content/drive/MyDrive/Samples'
COMMENTS_URL = 'http://www.nytimes.com/svc/community/V3/requestHandler?&cmd=GetCommentsAll&url='


files = []
for (root,dirs,file) in os.walk(samples_dir):
  files = file


for file in files:
  with open(samples_dir+'/'+file,'r') as sample:
    data = json.load(sample)
    anchor = data['response']['docs'][0]

    main = anchor['headline']['main']
    abstract = anchor['abstract']
    web_url = anchor['web_url']

    kwds = anchor['keywords']
    kwds_str = []
    for kwd in kwds:
      kwds_str.append(kwd['value'])

    response = requests.get(COMMENTS_URL+web_url).json()
    raw_comments = []
    for comment in response['results']['comments']:
        raw_comments.append(comment['commentBody'])

    if raw_comments != []:
      output = {
          "main":main,
          "abstract":abstract,
          "keywords":kwds_str,
          "comments":raw_comments,
          "web_url": web_url
      }

      with open(f'{output["main"]}.json','w',encoding='utf-8') as output_file:
        output_file.write(json.dumps(output,indent=4))
    else:
      continue